In [2]:
import pandas as pd
from datasets import load_dataset
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import unicodedata
import string
from tokenizers import ByteLevelBPETokenizer
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

c:\Users\Pedro\Documents\GitHub\Back-DomRock\Back-DomRock\Back-DomRock\ambvirtual\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Carregando e lendo o datasets:

In [2]:
dataset = load_dataset("ruanchaves/b2w-reviews01", split='train')
df = pd.DataFrame(dataset)
df.to_csv('./datasets/dataset.csv', index=False)


In [ ]:
df.head()

Extraindo apenas as colunas que serão utilizadas: reviewer_id', 'product_name', 'recommend_to_a_friend', 'review_text'

In [7]:
import pandas as pd

def extrair_coluna(arquivo_entrada, colunas_desejadas, arquivo_saida):
    df = pd.read_csv(arquivo_entrada)

    # Adiciona "Nota:" antes das notas
    df['overall_rating'] = df['overall_rating'].apply(lambda x: f'Nota: {x}')

    # Extrai as colunas desejadas
    dados_coluna = df[colunas_desejadas]

    # Salva os dados em um novo arquivo CSV
    dados_coluna.to_csv(arquivo_saida, index=False)

arquivo_entrada = "./datasets/dataset.csv"
colunas_desejadas = ['product_name', 'overall_rating', 'site_category_lv1', 'site_category_lv2', 'review_text']
novo_arquivo = "./datasets/dataset_reduzido.csv"

extrair_coluna(arquivo_entrada, colunas_desejadas, novo_arquivo)


C:\Users\Pedro\AppData\Local\Temp\ipykernel_10080\3566867852.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo_entrada)


Prepocessamento:

In [4]:
arquivo_entrada = "./datasets/dataset_reduzido.csv"
df = pd.read_csv(arquivo_entrada)

def preprocess_text(text):
    # Convertendo letras maiúculas para minúsculas.
    text = text.lower()
    
    # Removendo acentos.
    text = ''.join(
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
    )
    # Removendo pontuações.
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Removendo stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text


stop_words = set(stopwords.words('portuguese'))
# Lista de colunas que serão pré-processadas
colunas_texto = ['product_name', 'overall_rating', 'site_category_lv1', 'site_category_lv2', 'review_text']


# Aplicando o pré-processamento em cada coluna de texto
for coluna in colunas_texto:
    df[coluna] = df[coluna].apply(lambda x: preprocess_text(str(x)))

# Salvando o DataFrame pré-processado em um novo arquivo CSV
df.to_csv('./datasets/dataset_preprocessed.csv', index=False)

In [ ]:
df = pd.read_csv('./datasets/dataset_preprocessed.csv')

# Obter os valores e contagem
values = df['overall_rating'].value_counts()

# Criar o gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(values.index, values.values)

# Configurar os labels e título
plt.xlabel('Rating')
plt.ylabel('Contagem')
plt.title('Distribuição de Recomendações')

# Exibir o gráfico
plt.show()

In [1]:
import pandas as pd
arquivo_entrada = "./datasets/dataset_amostra.csv"
novo_arquivo = "./datasets/dataset_preprocessed.csv"

def extrair_coluna_e_agrupar(arquivo_entrada, arquivo_saida):
    df = pd.read_csv(arquivo_entrada)

    # Remove entradas com valores nulos em colunas relevantes
    df = df.dropna(subset=['product_name', 'overall_rating', 'site_category_lv1', 'site_category_lv2', 'review_text','reviewer_gender','reviewer_state'])

    # Adiciona "Nota:" antes das notas
    df['overall_rating'] = df['overall_rating'].apply(lambda x: f'Nota: {x}')

    # Agrupa os produtos pelo nome e concatena as notas e reviews
    dados_agregados = df.groupby('product_name').agg({
        'overall_rating': 'first',
        'site_category_lv1': 'first',
        'site_category_lv2': 'first',
        'reviewer_gender': 'first',
        'reviewer_state': 'first',
        'review_text': lambda x: ' | '.join(x)
    }).reset_index()

    # Salva os dados agrupados em um novo arquivo CSV
    dados_agregados.to_csv(arquivo_saida, index=False)

extrair_coluna_e_agrupar(arquivo_entrada, novo_arquivo)


Extrair DATASET para testes:


In [19]:
import pandas as pd

arquivo_entrada = "./datasets/dataset_preprocessed.csv"
novo_arquivo = "./datasets/dataset_amostra.csv"
porcentagem_amostra = 0.2  # 20%

def reduzir_amostra(arquivo_entrada, arquivo_saida, porcentagem_amostra):
    df = pd.read_csv(arquivo_entrada)

    # Seleciona uma amostra aleatória de 20% das linhas
    amostra = df.sample(frac=porcentagem_amostra, random_state=42)  # random_state para reprodutibilidade

    # Salva a amostra em um novo arquivo CSV
    amostra.to_csv(arquivo_saida, index=False)

reduzir_amostra(arquivo_entrada, novo_arquivo, porcentagem_amostra)


Fazendo um CSV com a coluna do Sexo e Região para testes

In [17]:
import pandas as pd

def extrair_coluna(arquivo_entrada, colunas_desejadas, arquivo_saida):
    df = pd.read_csv(arquivo_entrada)

    # Adiciona "Nota:" antes das notas
    df['overall_rating'] = df['overall_rating'].apply(lambda x: f'Nota: {x}')

    # Extrai as colunas desejadas
    dados_coluna = df[colunas_desejadas]

    # Salva os dados em um novo arquivo CSV
    dados_coluna.to_csv(arquivo_saida, index=False)

arquivo_entrada = "./datasets/dataset.csv"
colunas_desejadas = ['product_name', 'overall_rating', 'site_category_lv1', 'site_category_lv2', 'review_text','reviewer_gender','reviewer_state']
novo_arquivo = "./datasets/dataset_reduzido.csv"

extrair_coluna(arquivo_entrada, colunas_desejadas, novo_arquivo)


C:\Users\Pedro\AppData\Local\Temp\ipykernel_10868\1329039899.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo_entrada)


In [18]:


arquivo_entrada = "./datasets/dataset_reduzido.csv"
df = pd.read_csv(arquivo_entrada)

def preprocess_text(text):
    # Convertendo letras maiúculas para minúsculas.
    text = text.lower()
    
    # Removendo acentos.
    text = ''.join(
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
    )
    # Removendo pontuações.
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Removendo stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text


stop_words = set(stopwords.words('portuguese'))
# Lista de colunas que serão pré-processadas
colunas_texto = ['product_name', 'overall_rating', 'site_category_lv1', 'site_category_lv2', 'review_text','reviewer_gender','reviewer_state']


# Aplicando o pré-processamento em cada coluna de texto
for coluna in colunas_texto:
    df[coluna] = df[coluna].apply(lambda x: preprocess_text(str(x)))

# Salvando o DataFrame pré-processado em um novo arquivo CSV
df.to_csv('./datasets/dataset_preprocessed.csv', index=False)
